# Домашнее задание к лекции «Продвинутый pandas 2»

In [49]:
import pandas as pd

## Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. 

Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. 

Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [50]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head(2)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


In [51]:
ratings['date'] = pd.to_datetime(ratings['timestamp'], unit='s')
ratings.head(2)

,userId,movieId,rating,timestamp,date
0,1,31,2.5,1260759144,2009-12-14 02:52:24
1,1,1029,3.0,1260759179,2009-12-14 02:52:59


In [52]:
lifetime = ratings.pivot_table(index = 'userId', values = 'date', 
                    aggfunc = ['min', 'max'], fill_value='').reset_index()
lifetime.head()

,userId,min,max
,,date,date
0,1,2009-12-14 02:51:48,2009-12-14 02:53:25
1,2,1996-06-21 11:09:55,1996-06-21 11:24:06
2,3,2011-02-28 02:53:09,2011-02-28 22:39:47
3,4,2000-02-05 19:25:14,2000-02-08 03:57:54
4,5,2006-11-12 23:10:44,2006-11-12 23:45:45


In [53]:
lifetime['life'] = lifetime['max'] - lifetime['min']
lifetime.head()

,userId,min,max,life
,,date,date,
0,1,2009-12-14 02:51:48,2009-12-14 02:53:25,0 days 00:01:37
1,2,1996-06-21 11:09:55,1996-06-21 11:24:06,0 days 00:14:11
2,3,2011-02-28 02:53:09,2011-02-28 22:39:47,0 days 19:46:38
3,4,2000-02-05 19:25:14,2000-02-08 03:57:54,2 days 08:32:40
4,5,2006-11-12 23:10:44,2006-11-12 23:45:45,0 days 00:35:01


In [54]:
lifetime.mean()[['life']]

life      201 days 02:01:01.004470
dtype: object

In [55]:
# Вариант 2. Через agg 
# Все-равно получилось три строчки(;-(

lifetime_mean = ratings.groupby('userId').agg([min, max])
lifetime_mean['life'] = (lifetime_mean['date']['max'] - lifetime_mean['date']['min'])
lifetime_mean.mean()[['life']]

life      201 days 02:01:01.004470
dtype: object

## Задание 2
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). 

Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

In [56]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [57]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [58]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [59]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [60]:
transport = rzd.merge(auto, how='outer').merge(air, how='outer')
transport

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [61]:
transport_address = transport.merge(client_base, how='left')
transport_address

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


## Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
1. У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
2. Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
3. Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

In [ ]:
''' 
Извиняюсь, немного подзавис на этом задании, а вы просили этого не делать!
Понял так, что необходимо описать ход действий.

Скорее всего начал бы решать с конца!
Сначала получить 'типичные' координаты пользователя (user_id), т.е. от куда он заходит чаще всего.
Для этого можно определить по координатам город (напр. модуль reverse_geocoder, см. ниже), это уже не просто числа - вопрос 3.
Далее собрать пользователей в сводную с расчетом моды по городу - вопрос 2.
После этого сгруппировать таблицы и объеденить таблицу визитов с покупками внешним соединением 
(или левым - нельзя купить без визита!) по пользователю.
И остается только объединить получившуюся таблицу с таблицей местоположения пользователей (левым соединением) - вопрос 1.

'''

In [ ]:
#!pip install reverse_geocoder

In [62]:
import reverse_geocoder as rg

In [63]:
coordinates = (59.939792, 30.314472)

In [64]:
result = rg.search(coordinates)
result

[OrderedDict([('lat', '59.93863'),
              ('lon', '30.31413'),
              ('name', 'Saint Petersburg'),
              ('admin1', 'St.-Petersburg'),
              ('admin2', ''),
              ('cc', 'RU')])]

In [65]:
result[0]['name']

'Saint Petersburg'